# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import pyopia.classify
import pyopia.process
import pyopia.pipeline
import pyopia.statistics
import pyopia.instrument.silcam

import exampledata

# Load an image

In [ ]:
# define the image name for loading and later use in extracting timestamp
# this image example can be dowloaded from here:
#     https://pysilcam.blob.core.windows.net/test-data/D20181101T142731.838206.silc

filename = exampledata.get_example_silc_image()

# load a silcam image and an example
img = np.load(filename, allow_pickle=False)

In [ ]:
# show the image
plt.imshow(img)

# Initialise the CNN

In [ ]:
# load the model (this example can be found here:
#    https://github.com/SINTEF/PySilCam/wiki/ml_models/keras_model.zip)

model_path = exampledata.get_example_model()

steps = {'classifier': pyopia.classify.Classify(model_path=model_path),
         'load': pyopia.instrument.silcam.SilCamLoad(),
         'imageprep': pyopia.instrument.silcam.ImagePrep(image_level='imraw'),
         'segmentation': pyopia.process.Segment(threshold=0.85),
         'statextract': pyopia.process.CalculateStats()}

# Calculate particle statistics

In [ ]:
# Initialise the pipeline and run the initial steps
processing_pipeline = pyopia.pipeline.Pipeline(steps)

# Load an image (from the test suite)
filename = exampledata.get_example_silc_image()

# Process the image to obtain the stats dataframe
stats = processing_pipeline.run(filename)

## The STATS DataFrame

In [ ]:
# print the stats DataFrame
stats

# Analysis, statistics and plotting

In [ ]:
# there are lots of functions in pyopia.statistics. Here is just an example.

# Calculate the volume distribution from the stats DataFrame.
# (Usually several images would be needed for statistics to converge.
# This can be done by appending new image stats to the DataFrame)
dias, vd = pyopia.statistics.vd_from_stats(stats, 24)

# plot the volume distribution
plt.plot(dias, vd)
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Distribution [uL/sample vol.]')

In [ ]:
stats.to_xarray()

In [ ]:
raise
import pyopia.io

In [ ]:
datafilename = 'proc/test'
steps_string = pyopia.pipeline.steps_to_string(steps)

In [ ]:
from datetime import datetime
import h5py
import pandas as pd
from pyopia import __version__ as pyopia_version


with h5py.File(datafilename + '-STATS.h5', "a") as fh:
    meta = fh.require_group('Meta')
    meta.attrs['Modified'] = str(datetime.now())
    meta.attrs['PyOpia version'] = pyopia_version
    meta.attrs['Pipeline steps'] = steps_string

In [ ]:
with h5py.File(datafilename + '-STATS.h5', "r") as f:
    keys = list(f['Meta'].attrs.keys())
    steps_fromh5 = f['Meta'].attrs['Pipeline steps']
steps_fromh5

In [ ]:
data = pd.read_hdf(datafilename + '-STATS.h5')
data

In [ ]:
loaded_stats = pyopia.io.show_h5_meta('proc/test-STATS.h5')